In [26]:
import numpy as np
from tensorflow.contrib import layers

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from sklearn import metrics
import tensorflow as tf
from tensorflow.contrib import learn

### Download and load MNIST data.

mnist = learn.datasets.load_dataset('mnist')

### Linear classifier.

feature_columns = learn.infer_real_valued_columns_from_input(mnist.train.images)
classifier = learn.TensorFlowLinearClassifier(
    feature_columns=feature_columns, n_classes=10, batch_size=100, steps=1000,
    learning_rate=0.01)
classifier.fit(mnist.train.images, mnist.train.labels)
score = metrics.accuracy_score(
    mnist.test.labels, classifier.predict(mnist.test.images))
print('Accuracy: {0:f}'.format(score))

### Convolutional network


def max_pool_2x2(tensor_in):
  return tf.nn.max_pool(
      tensor_in, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def conv_model(X, y):
  # pylint: disable=invalid-name,missing-docstring
  # reshape X to 4d tensor with 2nd and 3rd dimensions being image width and
  # height final dimension being the number of color channels.
  X = tf.reshape(X, [-1, 28, 28, 1])
  # first conv layer will compute 32 features for each 5x5 patch
  with tf.variable_scope('conv_layer1'):
    h_conv1 = learn.ops.conv2d(X, n_filters=32, filter_shape=[5, 5],
                               bias=True, activation=tf.nn.relu)
    h_pool1 = max_pool_2x2(h_conv1)
  # second conv layer will compute 64 features for each 5x5 patch.
  with tf.variable_scope('conv_layer2'):
    h_conv2 = learn.ops.conv2d(h_pool1, n_filters=64, filter_shape=[5, 5],
                               bias=True, activation=tf.nn.relu)
    h_pool2 = max_pool_2x2(h_conv2)
    # reshape tensor into a batch of vectors
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
  # densely connected layer with 1024 neurons.
  h_fc1 = learn.ops.dnn(
      h_pool2_flat, [1024], activation=tf.nn.relu, dropout=0.5)
  return learn.models.logistic_regression(h_fc1, y)

# Training and predicting.
classifier = learn.TensorFlowEstimator(
    model_fn=conv_model, n_classes=10, batch_size=100, steps=20000,
    learning_rate=0.001)
classifier.fit(mnist.train.images, mnist.train.labels)
score = metrics.accuracy_score(
    mnist.test.labels, classifier.predict(mnist.test.images))
print('Accuracy: {0:f}'.format(score))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


ValueError: Target's dtype should be int32, int64 or compatible. Instead got <dtype: 'uint8'>.

In [14]:
mnist_train_labels = mnist.train.labels.astype(np.int32)

In [15]:
feature_columns = learn.infer_real_valued_columns_from_input(mnist.train.images)
classifier = learn.TensorFlowLinearClassifier(
    feature_columns=feature_columns, n_classes=10, batch_size=100, steps=1000,
    learning_rate=0.01)
classifier.fit(mnist.train.images, mnist_train_labels)
score = metrics.accuracy_score(
    mnist.test.labels, classifier.predict(mnist.test.images))
print('Accuracy: {0:f}'.format(score))

Accuracy: 0.889200


In [31]:
def max_pool_2x2(tensor_in):
  return tf.nn.max_pool(
      tensor_in, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def conv_model(X, y):
  # pylint: disable=invalid-name,missing-docstring
  # reshape X to 4d tensor with 2nd and 3rd dimensions being image width and
  # height final dimension being the number of color channels.
  X = tf.reshape(X, [-1, 28, 28, 1])
  # first conv layer will compute 32 features for each 5x5 patch
  with tf.variable_scope('conv_layer1'):
    h_conv1 = tf.contrib.layers.conv2d(X, num_outputs=32, kernel_size=[5, 5])
    h_pool1 = max_pool_2x2(h_conv1)
  # second conv layer will compute 64 features for each 5x5 patch.
  with tf.variable_scope('conv_layer2'):
    h_conv2 = tf.contrib.layers.conv2d(h_pool1, num_outputs=64, kernel_size=[5, 5])
    h_pool2 = max_pool_2x2(h_conv2)
    # reshape tensor into a batch of vectors
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    
  # densely connected layer with 1024 neurons.
  normalizer_fn = layers.dropout
  normalizer_params = {'keep_prob': 0.5}
  h_fc1 = layers.stack(
      h_pool2_flat, layers.fully_connected, [1024],
        normalizer_fn=normalizer_fn,
        normalizer_params=normalizer_params)
    
  return learn.models.logistic_regression(h_fc1, y)

# Training and predicting.
classifier = learn.TensorFlowEstimator(
    model_fn=conv_model, n_classes=10, batch_size=100, steps=5000,
    learning_rate=0.001)

import logging
logging.getLogger().setLevel(logging.INFO)

classifier.fit(mnist.train.images, mnist.train.labels)
score = metrics.accuracy_score(
    mnist.test.labels, classifier.predict(mnist.test.images))
print('Accuracy: {0:f}'.format(score))

INFO:tensorflow:Create CheckpointSaver
INFO:tensorflow:Step 1: loss = 2.57975
INFO:tensorflow:Step 101: loss = 2.50789
INFO:tensorflow:Step 201: loss = 2.27466
INFO:tensorflow:Saving checkpoints for 300 into /tmp/tmpY9kvC1/model.ckpt.
INFO:tensorflow:Step 301: loss = 2.11321
INFO:tensorflow:Step 401: loss = 1.99002
INFO:tensorflow:Step 501: loss = 1.76649
INFO:tensorflow:Saving checkpoints for 600 into /tmp/tmpY9kvC1/model.ckpt.
INFO:tensorflow:Step 601: loss = 1.43426
INFO:tensorflow:Step 701: loss = 1.27391
INFO:tensorflow:Step 801: loss = 1.00421
INFO:tensorflow:Saving checkpoints for 900 into /tmp/tmpY9kvC1/model.ckpt.
INFO:tensorflow:Step 901: loss = 0.733193
INFO:tensorflow:Step 1001: loss = 0.686502
INFO:tensorflow:Step 1101: loss = 0.725624
INFO:tensorflow:Saving checkpoints for 1200 into /tmp/tmpY9kvC1/model.ckpt.
INFO:tensorflow:Step 1201: loss = 0.590115
INFO:tensorflow:Step 1301: loss = 0.603186
INFO:tensorflow:Step 1401: loss = 0.428882
INFO:tensorflow:Saving checkpoints f

Accuracy: 0.930000
